# `fho_main.json` Data Exploration

In [ ]:
# required imports
import json
import math
import re
import string

import matplotlib.pyplot as plt

%matplotlib inline

First load `fho_main.json`.

In [ ]:
with open("/nfs/turbo/coe-chaijy/datasets/ego4d/v2/annotations/fho_main.json") as f:
    fho_main = json.load(f)

Let's verify if `is_rejected` is correlated to `is_valid_action`.

In [ ]:
for video in fho_main["videos"]:
    for interval in video["annotated_intervals"]:
        for action in interval["narrated_actions"]:
            if action["is_valid_action"] == action["is_rejected"]:
                # if we're here, it means either 1. it's a valid action but
                # not rejected, or 2. it's not a valid action but it's accepted.
                print(f'is_valid_action: {action["is_valid_action"]}')
                print(f'is_rejected: {action["is_rejected"]}')
                print(f'uid: {action["uid"]}')
                print(f'narration_text: {action["narration_text"]}')
                print(f'video_uid: {video["video_uid"]}')
                print(f'clip_uid: {interval["clip_uid"]}')
                print("========================================================")

How about actions that are not rejected, prefixed by `#C` but not valid?

In [ ]:
for video in fho_main["videos"]:
    for interval in video["annotated_intervals"]:
        for action in interval["narrated_actions"]:
            if (
                not action["is_rejected"]
                and action["narration_text"].startswith("#C")
                and not action["is_valid_action"]
            ):
                print(f'is_valid_action: {action["is_valid_action"]}')
                print(f'is_rejected: {action["is_rejected"]}')
                print(f'uid: {action["uid"]}')
                print(f'narration_text: {action["narration_text"]}')
                print(f'video_uid: {video["video_uid"]}')
                print(f'clip_uid: {interval["clip_uid"]}')
                print("========================================================")

Is `(start_sec, end_sec)` same as `(clip_start_sec, clip_end_sec)`?

In [ ]:
for video in fho_main["videos"]:
    for interval in video["annotated_intervals"]:
        for action in interval["narrated_actions"]:
            if not math.isclose(
                action["start_sec"], action["clip_start_sec"], rel_tol=1e-5
            ) or not math.isclose(
                action["end_sec"], action["clip_end_sec"], rel_tol=1e-5
            ):
                print(f'start_sec: {action["start_sec"]}')
                print(f'clip_start_sec: {action["clip_start_sec"]}')
                print(f'end_sec: {action["end_sec"]}')
                print(f'clip_end_sec: {action["clip_end_sec"]}')
                print(f'uid: {action["uid"]}')
                print(f'narration_text: {action["narration_text"]}')
                print(f'video_uid: {video["video_uid"]}')
                print(f'clip_uid: {interval["clip_uid"]}')
                print("========================================================")

`(start_sec, end_sec)` and `(clip_start_sec, clip_end_sec)` are not the same. The former denotes the times from the full video, while the latter denotes the times from clips.

How long are actions?

In [ ]:
action_times = []
for video in fho_main["videos"]:
    for interval in video["annotated_intervals"]:
        for action in interval["narrated_actions"]:
            action_time = action["clip_end_sec"] - action["clip_start_sec"]
            if action_time < 3:
                print(f'start_frame: {action["start_frame"]}')
                print(f'end_frame: {action["end_frame"]}')
                print(f'uid: {action["uid"]}')
                print(f'narration_text: {action["narration_text"]}')
                print(f'video_uid: {video["video_uid"]}')
                print(f'clip_uid: {interval["clip_uid"]}')
                print("========================================================")
            action_times.append(action_time)

n, bins, patches = plt.hist(action_times)

# Annotate the frequency above each bar
for i in range(len(n)):
    plt.annotate(
        f"{n[i]:.0f}",
        xy=((bins[i] + bins[i + 1]) / 2, n[i]),
        xytext=(0, 5),
        textcoords="offset points",
        ha="center",
        va="bottom",
    )

# Add labels and a title
plt.xlabel("Seconds")
plt.ylabel("Number of Actions")
plt.title("Action Duration")

# Display the plot
plt.show()

Do all `narrated_text`s end with a period?

In [ ]:
punc_counts = {p: 0 for p in string.punctuation}
for video in fho_main["videos"]:
    for interval in video["annotated_intervals"]:
        for action in interval["narrated_actions"]:
            last_char = action["narration_text"][-1]
            if last_char in punc_counts:
                punc_counts[last_char] += 1

filtered_counts = {k: v for k, v in punc_counts.items() if v > 0}

# Create a bar graph
bars = plt.bar(filtered_counts.keys(), filtered_counts.values())

# Add labels and a title
plt.xlabel("Punctuation")
plt.ylabel("Frequency")
plt.title("Frequency of Sentence Endings with Punctuation")

# Annotate the count above each bar
for bar in bars:
    plt.annotate(
        f"{bar.get_height():.0f}",
        xy=(bar.get_x() + bar.get_width() / 2, bar.get_height()),
        xytext=(0, 3),
        textcoords="offset points",
        ha="center",
        va="bottom",
    )

# Display the plot
plt.show()

Any `#summary`s?

In [ ]:
SUMMARY_REGEX = re.compile(r"\#summary", re.IGNORECASE)
for video in fho_main["videos"]:
    for interval in video["annotated_intervals"]:
        for action in interval["narrated_actions"]:
            if SUMMARY_REGEX.search(action["narration_text"]):
                print(f'uid: {action["uid"]}')
                print(f'narration_text: {action["narration_text"]}')
                print(f'video_uid: {video["video_uid"]}')
                print(f'clip_uid: {interval["clip_uid"]}')
                print("========================================================")

Is `#unsure` always at the end?

In [ ]:
UNSURE_REGEX = re.compile(r"\#unsure", re.IGNORECASE)
ENDS_WITH_UNSURE_REGEX = re.compile(r"\#unsure$", re.IGNORECASE)
for video in fho_main["videos"]:
    for interval in video["annotated_intervals"]:
        for action in interval["narrated_actions"]:
            if UNSURE_REGEX.search(
                action["narration_text"]
            ) and not ENDS_WITH_UNSURE_REGEX.search(action["narration_text"].strip()):
                print(f'uid: {action["uid"]}')
                print(f'narration_text: {action["narration_text"]}<|eos|>')
                print(f'video_uid: {video["video_uid"]}')
                print(f'clip_uid: {interval["clip_uid"]}')
                print("========================================================")